<a href="https://colab.research.google.com/github/anu04596/DeepLearningTuotorial/blob/main/Day12(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building Fully Connected Neural Netwrok from scratch

Import Libraries

In [3]:
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import random

Load Dataset

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using", device)

Using cuda


In [4]:
tranform=transforms.ToTensor()
train_dataset=torchvision.datasets.MNIST(root='./data',train=True,transform=tranform,download=True)
test_dataset=torchvision.datasets.MNIST(root='./data',train=False,transform=tranform,download=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 16.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 489kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.51MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.78MB/s]


In [5]:
subset_size = int(0.1 * len(train_dataset))
indices = random.sample(range(len(train_dataset)), subset_size)
small_train_dataset = Subset(train_dataset, indices)
small_train_loader = DataLoader(small_train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [9]:
print(len(small_train_dataset))
print(len(test_loader))

6000
157


Multi Layer Perceptron

In [16]:
class MLP(torch.nn.Module):
  def __init__(self):
    super(MLP,self).__init__()
    self.w1=torch.nn.Parameter(torch.randn(784,128)*0.01)
    self.b1=torch.nn.Parameter(torch.zeros(128))

    self.w2=torch.nn.Parameter(torch.randn(128,64)*0.01)
    self.b2=torch.nn.Parameter(torch.zeros(64))

    self.w3=torch.nn.Parameter(torch.randn(64,10)*0.01)
    self.b3=torch.nn.Parameter(torch.zeros(10))

  def forward(self,x):
    x=x.view(-1,784)
    h1=F.relu(x @ self.w1 + self.b1)
    h2=F.relu(h1 @ self.w2 + self.b2)
    output=h2 @ self.w3 + self.b3
    return output

model=MLP().to(device)

Training

In [18]:
optimizer=torch.optim.Adam(model.parameters(), lr=1e-3)
loss=torch.nn.CrossEntropyLoss()

for epoch in range(5):
  model.train()
  total_loss=0

  for images,labels in small_train_loader:
    images,labels=images.to(device),labels.to(device)

    optimizer.zero_grad()
    output=model(images)
    loss_value=loss(output,labels)
    loss_value.backward()
    optimizer.step()

    total_loss+=loss_value.item()

  print(f"Epoch {epoch+1}, Loss: {total_loss / len(small_train_loader):.4f}")

Epoch 1, Loss: 0.4585
Epoch 2, Loss: 0.3762
Epoch 3, Loss: 0.3244
Epoch 4, Loss: 0.2901
Epoch 5, Loss: 0.2569


Evaluate

In [19]:
model.eval()
correct=0
total=0

with torch.no_grad():
  for images, labels in test_loader:
    images, labels= images.to(device), labels.to(device)
    outputs=model(images)
    _,predicted=torch.max(outputs,1)
    correct+=(predicted==labels).sum().item()
    total+=labels.size(0)

print(f"Accuracy: {100* correct/total:.2f}%")

Accuracy: 90.95%
